If you haven't yet, start by setting up your environment and datasets by following the instructions in the README. It should be something like:
* `make create_environment`
* `conda activate covid_nlp`
* `make update_environment`
* `make data`

Several common packages that you may want to use (e.g. UMAP, HDBSCAN, enstop, sklearn) have already been added to the `covid_nlp` environment via `environment.yml`. To add more, edit that file and do a:
  ` make update_environment`

## Document embedding of abstracts
In this notebook we'll follow https://github.com/gclen/covid19-kaggle/blob/master/embed_abstracts_interactive.ipynb to embed abstracts. It uses the the techniques from https://umap-learn.readthedocs.io/en/latest/document_embedding.html to embed documents using UMAP. 

In [ ]:
# Quick cell to make jupyter notebook use the full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Automatically pick up code changes in the `src` module
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd

In [ ]:
# Useful imports from easydata
from src import paths
from src.data import Dataset
from src import workflow

In [ ]:
# other packages
from scipy import sparse

# tokenizing/vectorizing
import en_core_sci_sm # A full spaCy pipeline for biomedical data.
from scispacy.custom_tokenizer import combined_rule_tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
# dimension reduction
import umap
import umap.plot
# clustering
import hdbscan

In [ ]:
# Some plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import show, save, output_notebook, output_file
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

## Load up the dataset

The metadata has been augmented with where the files can be found relative to `paths["interim_data_path"]`

In [ ]:
#paths['interim_data_path']

In [ ]:
workflow.available_datasets()

If the previous cell returned an empty list, go back and re-run `make data` as described at the top of this notebook.

In [ ]:
ds_name = 'covid_nlp_20200319'

In [ ]:
# Load the dataset
meta_ds = Dataset.load(ds_name)

In [ ]:
print(meta_ds.DESCR[:457])

In [ ]:
# The processed dataframe is the `data` method of this data source 
meta_df = meta_ds.data
meta_df.head()

In [ ]:
# optionally filter it down to published papers with a cc-by license

meta_df.file_type.value_counts()

In [ ]:
meta_df = meta_df[(meta_df.file_type=='comm_use_subset') | (meta_df.file_type=='noncomm_use_subset')].copy()

## Basics on the dataset

The JSON files given in the `path` column of the metadata dataframe are the papers in `json` format (as dicts)
that include the following keys:
* `paper_id`
* `metadata`
* `abstract`
* `body_text`
* `bib_entries`
* `ref_entries`
* `back_matter`

where the `paper_id` is the sha hash from the medadata.

For example:

In [ ]:
filename = paths['interim_data_path'] / ds_name / meta_df['path'][0]
file = json.load(open(filename, 'rb'))
file.keys()

# Embedding abstracts

In [ ]:
abstracts = meta_df.abstract.dropna()

In [ ]:
abstracts[:5]

In [ ]:
len(abstracts)

### Vectorize the abstracts (initial embedding)

#### Set up tokenizer

We'll use scispacy

In [ ]:
nlp = en_core_sci_sm.load()
nlp.tokenizer = combined_rule_tokenizer(nlp)

In [ ]:
# We just want the tokenization !
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')
nlp.remove_pipe('ner')

In [ ]:
def spacy_tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]

#### Vectorize to get word-document matrix

In [ ]:
%%time
vectorizer = TfidfVectorizer(min_df=5, tokenizer=spacy_tokenizer)
word_doc_matrix = vectorizer.fit_transform(abstracts)

In [ ]:
word_doc_matrix

In [ ]:
save_path = paths['processed_data_path'] / "spacy-abstract-matrix-cc-by.npz"
#save_path = paths['processed_data_path'] / "spacy-abstract-matrix.npz"

In [ ]:
#sparse.save_npz(save_path, word_doc_matrix)

## XXX Split the notebook here eventually 

In [ ]:
word_doc_matrix = sparse.load_npz(save_path)

### Reduce dimension using UMAP and Hellinger distance


In [ ]:
%%time
embedding_2d = umap.UMAP(n_components=2, n_neighbors=10,
                         metric='hellinger', init='random',
                         random_state=42).fit(word_doc_matrix)

### XXX Replace with vectorizer code  to compare

### XXX EM?

Shorten abstracts for display

In [ ]:
short_abstracts = [a[:140] for a in abstracts]

In [ ]:
hover_df = meta_df[meta_df.abstract_length > 0].reset_index()
hover_df['short_abstracts'] = short_abstracts

#### Cluster

In [ ]:
%%time
clusterer = hdbscan.HDBSCAN(min_cluster_size=15)
clusterer.fit_predict(embedding_2d.embedding_)
labels = clusterer.labels_

In [ ]:
hover_df['cluster'] = labels

In [ ]:
hover_cols = ['title', 'file_type', 'short_abstracts', 'cluster']

In [ ]:
f = umap.plot.interactive(embedding_2d, labels=hover_df['cluster'],
                          hover_data=hover_df.reset_index()[hover_cols]);
show(f)

### Rank points based on distance to a representative point

In [ ]:
from src.utils import RankedPoints

In [ ]:
examples = RankedPoints(embedding_2d.embedding_, clusterer, metric='euclidean')

In [ ]:
examples.calculate_all_distances_to_center()
examples.get_all_cluster_rankings()

In [ ]:
hover_df['rank_in_cluster'] = examples.embedding_df['rank_in_cluster']

In [ ]:
top_5_papers = {}
grouped_by_cluster = hover_df.groupby('cluster')

for cluster_id, group in grouped_by_cluster:
    top_papers = group.sort_values('rank_in_cluster', ascending=True).head(5)
    
    top_5_papers[int(cluster_id)] = '<ol>' + ''.join([f'<li><a href="{r.doi}">{r.title}</a></li>' for _, r in top_papers.iterrows()]) + '</ol>'

In [ ]:
top_5_papers[0]

<ol><li><a href="http://dx.doi.org/10.1371/journal.pbio.0030172">A Three-Stemmed mRNA Pseudoknot in the SARS Coronavirus Frameshift Signal</a></li><li><a href="http://dx.doi.org/10.1093/nar/gkw480">A novel role for poly(C) binding proteins in programmed ribosomal frameshifting</a></li><li><a href="http://dx.doi.org/10.1093/nar/gkr686">mRNA pseudoknot structures can act as ribosomal roadblocks</a></li><li><a href="http://dx.doi.org/10.1093/nar/gkl1033">Identification of functional, endogenous programmed −1 ribosomal frameshift signals in the genome of Saccharomyces cerevisiae</a></li><li><a href="http://dx.doi.org/10.1371/journal.pone.0062283">Regulation of Programmed Ribosomal Frameshifting by Co-Translational Refolding RNA Hairpins</a></li></ol>

### This can now be used as input to an interactive plot

See https://github.com/gclen/covid19-kaggle for an example